In [1]:
import math
import random

class Player:
    def __init__(self, letter):
        #Letter initialized as either 'x' or 'o'
        self.letter = letter
        
    #We want all players get their next move in the game
    def get_move(self, game):
        pass
    
class RandomComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        
    def get_move(self, game):
        square = random.choice(game.available_moves())
        return square


class HumanPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + " 's turn. Input move (0-8): ")
            #we are going to check that this is a correct value by trying to cast
            # it to an integer, and if it is not then we say it is invalid
            # If that spot is not available on the board then we also it is invalid
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            
            except ValueError:
                print('Invalid Square. Try Again!!')
                
        return val     
    
class GeniusComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        
    def get_move(self, game):
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves()) #Randomly Choose One
        else:
            #Get the square based off the minimax algorithm
            square = self.minimax(game, self.letter)['position']
        return square
    
    def minimax(self, state, player):
        max_player = self.letter #This is us as we want to maximize our score
        other_player = 'O' if player =='X' else 'X'
        # First, we want to check if the previous move is a winner
        # This is our best case
        if state.current_winner == other_player:
            #We should return position and score because we need to keep
            #track of the score for minimax to work
            return {'position': None,
                    'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else (
                            -1 * (state.num_empty_squares() + 1))
                    }
            
        elif not state.empty_squares(): #No empty stones
            return {'position': None, 'score': 0 }
            
        if player == max_player:
            best = {'position': None, 'score': -math.inf}
            #Each scorre should be maximize (be larger)
        else:
            best = {'position': None, 'score': math.inf} # each score should minimize
            
        for possible_move in state.available_moves():
            
            # Step 1: make a move, try that spot
            state.make_move(possible_move, player)
            
            
            # Step 2: Recurse using minimax to simulate
            # a game after making that move
            sim_score = self.minimax(state, other_player) #Now we alternate player
            
            # Step 3: Undo the move
            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move #Otherwise this will get messed up in recursion part
            
            # Step 4: Update the Dictionaries if necessary
            # If the score is better than what we have at that time, then
            # we will update this weight to keep track of the 
            # best possible move to make
            if player == max_player:
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score #Replace best
            
        return best

In [14]:
import time

class TicTacToe:
    
    def __init__(self):
        #We will use single list to rep 3x3 board
        self.board = [' ' for _ in range(9)]
        self.current_winner = None #Keep Track of winnerr!
        
     
    def print_board(self):
        for row in [self.board[i*3:(i+1)*3]  for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')
    
    '''
        def print_board(self):
        for row in [self.board[i*3:(i+1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')
    '''
               
    @staticmethod
    def print_board_nums():
        # 0 | 1 | 2 etc
        #Tells us what nu,ber corrosponds to what box 
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')
            
    def available_moves(self):
        #Cleaner way 
        return [i for i, spot in enumerate(self.board) if spot == ' ']
        
        #Alternate way for writing the same thing
        '''moves = []
        for (i,spot) in enumerate(self.board):
            # ['x','x','o'] --> [(0,'x'),(1,'x'),(2,'o')]
            if spot == ' ':
                moves.append(i)
        return moves
        '''
        
    #Matched Everything till here    
    
    def empty_squares(self):
        return ' ' in self.board
    
    def num_empty_squares(self):
        #return len(self.available_moves())
        return self.board.count(' ')
    
    def make_move(self, square, letter):
        # If valid move, then make the move (assign square to letter) 
        # then return true, if invalid, return false
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False
    
    def winner(self, square, letter):
        #Winner if 3 in a row or column or diagonal
        row_ind = square//3
        row = self.board[row_ind*3 : (row_ind+1)*3]
        if all([spot == letter for spot in row]):
            return True
        
        #Check Column
        col_ind = square%3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True
        
        #Check Diagonal
        #0,2,4,6,8 - available diagonal elements
        if square %2 ==0 :
            diagonal1 = [self.board[i] for i in [0,4,8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            
            diagonal2 = [self.board[i] for i in [2,4,6]]
            if all([spot == letter for spot in diagonal2]):
                return True
            
        #If all above are false then this is the end
        return False
    
def play(game, x_player, o_player, print_game=True):
    if print_game:
        game.print_board_nums()
    
    #Starting Letter    
    letter = 'X'
    #Iterate while the game still has empty squares
    # We dont have to worry about the winner because we will just return that
    # which breaks the loop
    
    while game.empty_squares():
        #pass
        #After we made our move we need to alternate letters
            
        if letter == 'O':
            #square = o_player.get_move(game)
            square = o_player.get_move(game)
        else:
            #square = x_player.get_move(game)
            square = x_player.get_move(game)
            
        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square} ')
                game.print_board()
                print('') #Just empty line
            
            if game.current_winner:
                if print_game:
                    print(letter + ' wins!!')
                return letter
            
            letter = 'O' if letter == 'X' else 'X'  #Switches Player
            #if letter =='X':
                #letter = 'O'
            #else:
                #letter = 'X'
        #inducing a delay
        time.sleep(0.8)
            
    if print_game:
        print("It's a tie ");
        
if __name__ == '__main__':
    inpp='Y'
    while(inpp=='Y' or inpp=='y'):
        x_player = HumanPlayer('X')
        #o_player = RandomComputerPlayer('O')
        o_player = GeniusComputerPlayer('O')
        t = TicTacToe() #Instance of the TicTacToe
        play(t, x_player, o_player, print_game=True)
        print("Please Enter Y to play again or enter anything else to exit")
        inpp=input()[0]
        
    print('Thanks for playing')

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
X 's turn. Input move (0-8): 6
X makes a move to square 6 
|   |   |   |
|   |   |   |
| X |   |   |

O makes a move to square 4 
|   |   |   |
|   | O |   |
| X |   |   |

X 's turn. Input move (0-8): 
Invalid Square. Try Again!!
X 's turn. Input move (0-8): 8
X makes a move to square 8 
|   |   |   |
|   | O |   |
| X |   | X |

O makes a move to square 7 
|   |   |   |
|   | O |   |
| X | O | X |

X 's turn. Input move (0-8): 4
Invalid Square. Try Again!!
X 's turn. Input move (0-8): 2
X makes a move to square 2 
|   |   | X |
|   | O |   |
| X | O | X |

O makes a move to square 1 
|   | O | X |
|   | O |   |
| X | O | X |

O wins!!
Please Enter Y to play again or enter anything else to exit
no
Thanks for playing
